In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.datasets import load_digits
from sklearn import manifold
import numpy as np
import scipy
from sklearn.metrics.pairwise import rbf_kernel as rbf


In [2]:
X, y = load_digits(return_X_y=True)
X_red = manifold.SpectralEmbedding(n_components=2).fit_transform(X)

In [5]:
K_VALUES = [10, 20, 30, 40, 50, 60]
EPSILON_VALUES = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.8]

def _get_k_neighborhood(dataset, k, radius=False):
    if radius:
        neighbours = NearestNeighbors(radius=k, algorithm='ball_tree').fit(dataset)
        return neighbours

    neighbors = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(dataset)
    return neighbors

def _get_similarity_matrix(dataset):
    gamma = (1/np.var(pairwise_distances(dataset)))*0.1
    return rbf(dataset, gamma=gamma)


def get_KNN_precision(original, embedded, mode="NN"):
    result = []
    for k in K_VALUES:
        if mode == "NN":
            orig_neighbors  = _get_k_neighborhood(original, k)
            emb_neighbors  = _get_k_neighborhood(embedded, k)


            precision = 0
            for x,y in zip(original, embedded):

                x_neighbors = orig_neighbors.kneighbors(x.reshape(1,-1), return_distance=False)
                y_neighbors = emb_neighbors.kneighbors(y.reshape(1, -1), return_distance=False)

                precision += len(np.intersect1d(x_neighbors, y_neighbors))

            precision = precision / (k*len(original))

            result.append(precision)

        elif mode == "FN":
            orig_dist = pairwise_distances(original)
            emb_dist = pairwise_distances(embedded)

            precision=0
            for ind in range(len(original)):
                x_farthest = np.argpartition(orig_dist[ind], -k)[-k:]
                y_farthest = np.argpartition(emb_dist[ind], -k)[-k:]

                precision += len(np.intersect1d(x_farthest, y_farthest))

            precision = precision/ (k*(len(original)))

            result.append(precision)


        else:
            print(str(mode) + " is an invalid mode. Please select NN or FN")

    return result

def _get_epsilon_neighborhood_pr(dataset, embedded, latent=False):
    if latent:
        print("Warning Latent Embedding distances not built. Returning 0")
        return 0

    result_precision, result_recall = [], []
    print("Running Loop over Epsilon")
    for epsilon in EPSILON_VALUES:
        dataset_sim = _get_similarity_matrix(dataset)
        embedded_sim = _get_similarity_matrix(embedded)

        precision = 0
        recall = 0

        for ind in range(len(embedded)):

            d_mask = dataset_sim[ind] > epsilon
            emb_mask = embedded_sim[ind] > epsilon
            intersection = np.logical_and(d_mask, emb_mask)

            intersection_count = np.count_nonzero(intersection)
            d_count = np.count_nonzero(d_mask)
            emb_count = np.count_nonzero(emb_mask)
            

            if emb_count:
                precision += intersection_count/emb_count
            if d_count:
                recall += intersection_count/d_count

        precision = precision/len(embedded)
        recall = recall/len(embedded)

        result_precision.append(precision)
        result_recall.append(recall)
    return result_precision, result_recall

def get_original_pr(original, embedded):
    return _get_epsilon_neighborhood_pr(original, embedded)

def get_latent_pr(latent, embedded):
    return _get_epsilon_pr(latent, embedded, latent=True)

def interpoint_distance(x1, x2):
    net = 0
    for a,b in zip(x1, x2):
        net += norm(a-b)
    return net

In [87]:
jiwoon_matrix = euclidean2_np(X)

In [88]:
def _get_embedded_similarities(embedded):
    Y = embedded
    (n, d) = Y.shape
    print(d)
    sum_Y = np.sum(np.square(Y), 1)
    num = -2. * np.dot(Y, Y.T)
    num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
    num[range(n), range(n)] = 0.
    Q = num / np.sum(num)
    Q = np.maximum(Q, 1e-12)

    return Q

In [89]:
def _get_original_similarities(X):
    def Hbeta(D=np.array([]), beta=1.0):
        """
            Compute the perplexity and the P-row for a specific value of the
            precision of a Gaussian distribution.
        """

        # Compute P-row and corresponding perplexity
        P = np.exp(-D.copy() * beta)
        sumP = sum(P)
        H = np.log(sumP) + beta * np.sum(D * P) / sumP
        P = P / sumP
        return H, P


    def x2p(X=np.array([]), tol=1e-5, perplexity=30.0):
        """
            Performs a binary search to get P-values in such a way that each
            conditional Gaussian has the same perplexity.
        """

        # Initialize some variables
        print("Computing pairwise distances...")
        (n, d) = X.shape
        sum_X = np.sum(np.square(X), 1)
        D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
        P = np.zeros((n, n))
        beta = np.ones((n, 1))
        logU = np.log(perplexity)

        # Loop over all datapoints
        for i in range(n):

            # Print progress
            if i % 500 == 0:
                print("Computing P-values for point %d of %d..." % (i, n))

            # Compute the Gaussian kernel and entropy for the current precision
            betamin = -np.inf
            betamax = np.inf
            Di = D[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
            (H, thisP) = Hbeta(Di, beta[i])

            # Evaluate whether the perplexity is within tolerance
            Hdiff = H - logU
            tries = 0
            while np.abs(Hdiff) > tol and tries < 50:

                # If not, increase or decrease precision
                if Hdiff > 0:
                    betamin = beta[i].copy()
                    if betamax == np.inf or betamax == -np.inf:
                        beta[i] = beta[i] * 2.
                    else:
                        beta[i] = (beta[i] + betamax) / 2.
                else:
                    betamax = beta[i].copy()
                    if betamin == np.inf or betamin == -np.inf:
                        beta[i] = beta[i] / 2.
                    else:
                        beta[i] = (beta[i] + betamin) / 2.

                # Recompute the values
                (H, thisP) = Hbeta(Di, beta[i])
                Hdiff = H - logU
                tries += 1

            # Set the final row of P
            P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP

        # Return final P-matrix
        print("Mean value of sigma: %f" % np.mean(np.sqrt(1 / beta)))
        return P
    
    def pca(X=np.array([]), no_dims=50):
        """
            Runs PCA on the NxD array X in order to reduce its dimensionality to
            no_dims dimensions.
        """

        print("Preprocessing the data using PCA...")
        (n, d) = X.shape
        X = X - np.tile(np.mean(X, 0), (n, 1))
        (l, M) = np.linalg.eig(np.dot(X.T, X))
        Y = np.dot(X, M[:, 0:no_dims])
        return Y

    X = pca(X).real
    (n, d) = X.shape
    print("Shape is " + str(n) + " x " + str(d))
    P = x2p(X, 1e-5)
    
    P = P + np.transpose(P)
    print("sum is " + str(np.sum(P)))
    P = P / (2*n)
    
    P = np.maximum(P, 1e-12)

    return P
    




In [90]:
t = _get_original_similarities(X)

Preprocessing the data using PCA...
Shape is 1797 x 50
Computing pairwise distances...
Computing P-values for point 0 of 1797...
Computing P-values for point 500 of 1797...
Computing P-values for point 1000 of 1797...
Computing P-values for point 1500 of 1797...
Mean value of sigma: 11.630463
sum is nan


array([[1.00000000e-12, 1.00000000e-12, 8.38266386e-11, ...,
        2.17372644e-12, 2.83097629e-08, 2.69865422e-10],
       [1.00000000e-12, 1.00000000e-12, 4.74065879e-08, ...,
        1.11982657e-08, 1.48230458e-11, 2.79713310e-11],
       [8.38266386e-11, 4.74065879e-08, 1.00000000e-12, ...,
        2.00451022e-07, 5.41702556e-10, 1.84201288e-08],
       ...,
       [2.17372644e-12, 1.11982657e-08, 2.00451022e-07, ...,
        1.00000000e-12, 5.54146022e-10, 6.46634738e-06],
       [2.83097629e-08, 1.48230458e-11, 5.41702556e-10, ...,
        5.54146022e-10, 1.00000000e-12, 3.56913466e-08],
       [2.69865422e-10, 2.79713310e-11, 1.84201288e-08, ...,
        6.46634738e-06, 3.56913466e-08, 1.00000000e-12]])

In [93]:
def calculate_distances(X):
    N = X.shape[0]
    ss = np.sum(X**2, axis=1)
    dist = np.reshape(ss, [N, 1]) + np.reshape(ss, [1, N]) - 2*np.dot(X, X.T)
    dist = dist * np.asarray(dist>0,'float32')
    return dist



In [96]:
def _compute_original_similarities(X, sigma, metric, approxF=0):

    N = X.shape[0]
    sigma = np.full((1, 1797), sigma)
    if metric == 'euclidean':
        sqdistance = calculate_distances(X)
    elif metric == 'precomputed':
        sqdistance = X**2
    else:
        raise Exception('Invalid metric')
    euc_dist     = np.exp(-sqdistance / (np.reshape(2*(sigma**2), [N, 1])))
    np.fill_diagonal(euc_dist, 0.0 )

    if approxF > 0:
        sorted_euc_dist = euc_dist[:,:]
        np.sort(sorted_euc_dist, axis=1)
        row_sum = np.reshape(np.sum(sorted_euc_dist[:,1:approxF+1], axis=1), [N, 1])
    else:
        row_sum = np.reshape(np.sum(euc_dist, axis=1), [N, 1])

    return euc_dist/row_sum  # Possibly dangerous


In [129]:
def _compute_embedded_similarities(Y):
    N = Y.shape[0]
    sqdistance = calculate_distances(Y)
    one_over = 1./(sqdistance + 1)
    p_Yp_given_Y =  one_over/one_over.sum(axis=1).reshape((N, 1)) 
    return p_Yp_given_Y

In [130]:

EPSILON_VALUES = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.001]

def _get_epsilon_neighborhood_pr(dataset, embedded, latent=False):
    if latent:
        print("Warning Latent Embedding distances not built. Returning 0")
        return 0

    result_precision, result_recall = [], []

    dataset_sim = _compute_original_similarities(dataset, sigma=11.63, metric="euclidean")
    embedded_sim = _compute_embedded_similarities(embedded)

    for epsilon in EPSILON_VALUES:

        precision = 0
        recall = 0

        for ind in range(len(embedded)):

            d_mask = dataset_sim[ind] > epsilon
            emb_mask = embedded_sim[ind] > epsilon
            intersection = np.logical_and(d_mask, emb_mask)
            print(intersection)
            intersection_count = np.count_nonzero(intersection)
            d_count = np.count_nonzero(d_mask)
            emb_count = np.count_nonzero(emb_mask)

            if emb_count:
                precision += intersection_count/emb_count
            if d_count:
                recall += intersection_count/d_count

        precision = precision/len(embedded)
        recall = recall/len(embedded)

        result_precision.append(precision)
        result_recall.append(recall)
    return result_precision, result_recall

def get_original_pr(original, embedded):
    return _get_epsilon_neighborhood_pr(original, embedded)

def get_latent_pr(latent, embedded):
    return _get_epsilon_pr(latent, embedded, latent=True)

In [ ]:
get_original_pr(X, X_red)